<a href="https://colab.research.google.com/github/rezaBarzgar/SLR/blob/main/Bio_word_vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gensim
from gensim.models import KeyedVectors
from gensim.test.utils import datapath
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
!pip install pymed
import pymed
from pymed import PubMed
import pandas as pd
from pandas import DataFrame as df
import numpy as np

Downloadin data from figShare

In [ ]:
url = 'https://ndownloader.figshare.com/files/12551759'
wv_from_bin = KeyedVectors.load_word2vec_format(url, binary=True)

Testing the Model

In [ ]:
wv_from_bin.similarity('cold','conjunctivitis')

0.40380546

In [ ]:
wv_from_bin.most_similar(positive=['ketonuria', 'ketones'])

[('ketonemia', 0.8314430117607117),
 ('ketonemic', 0.7912544012069702),
 ('ketoacids', 0.7899306416511536),
 ('ketone', 0.7752881646156311),
 ('ketoacidaemia', 0.7740465402603149),
 ('ketonaemia', 0.7702386975288391),
 ('acetonuria', 0.7653355598449707),
 ('ketons', 0.7641968727111816),
 ('b-ketoacids', 0.7595287561416626),
 ('ketonuric', 0.7595204710960388)]

**Preprocess on Query**

In [ ]:
input_string = 'Ketonuria Excess of ketones present in urine Acetonuria (finding) Ketoacidurias Ketonaciduria Ketonurias Acetonurias'
input_string = input_string.lower()
stop_words = set(stopwords.words('english')) 
for ch in ['\\','`','*','_','{','}','[',']','(',')','>','#','+','.','!','$']:
  input_string = input_string.replace(ch,'')
string_list = input_string.split()
filtered_sentence = [w for w in string_list if not w in stop_words] 
filtered_sentence

['ketonuria',
 'excess',
 'ketones',
 'present',
 'urine',
 'acetonuria',
 'finding',
 'ketoacidurias',
 'ketonaciduria',
 'ketonurias',
 'acetonurias']

**Query expansion**

In [ ]:
string_list_2 = []
for item in filtered_sentence:
  if item in wv_from_bin.vocab:
    string_list_2.append(item)

similars = wv_from_bin.most_similar(positive=string_list_2, topn=30)

query = ''
for word in filtered_sentence:
  query =query + ' ' + word
for word in similars:
  if word[0] not in stop_words:
    query =query + ' ' + word[0]

query

' ketonuria excess ketones present urine acetonuria finding ketoacidurias ketonaciduria ketonurias acetonurias glucose-associated ketoacidosis-associated ketoacidosis-induced hyperaminoacidaemia ketonemia propionicacidemia hypercreatinaemia acetonemia investigationally result non-ketonuria introductionfunctional glycosuria addition hyperaminoacidemias hyperlability also uricacidemia propionic-acidemia possible hyperglyceroluria'

In [ ]:
def pubmed_search(query):
    pubmed = PubMed(tool="PubMedSearcher_reza", email="reza.barzgar99@gmail.com")

    search_term = query

    results = pubmed.query(search_term, max_results=200)
    articleList = []
    articleInfo = []

    for article in results:
        articleDict = article.toDict()
        articleList.append(articleDict)

    # Generate list of dict records which will hold all article details that could be fetch from PUBMED API
    for article in articleList:
        pubmedId = article['pubmed_id'].partition('\n')[0]
        # Append article info to dictionary
        articleInfo.append({u'pubmed_id': pubmedId,
                            u'title': article['title']})

    # Generate Pandas DataFrame from list of dictionaries
    articlesPD = pd.DataFrame.from_dict(articleInfo)
  
    return articlesPD.values.tolist()


**Search query in pubmed**

In [ ]:
query = query.replace(' ', ' OR ')

results = pubmed_search(query)
for item in results:
  print(item)

['33892525', 'Ovalbumin/lipopolysaccharide induced vasculitis in rats: a new predictive model.']
['33892523', 'Polymeric metal-containing ionic liquid sorbent coating for the determination of amines using headspace solid-phase microextraction.']
['33892519', 'Rational use of ultrasonography with triaging of patients to detect dengue plasma leakage in resource limited settings: a prospective cohort study.']
['33892518', 'Genetic landscape clustering of a large DNA barcoding dataset reveals shared patterns of genetic divergence among freshwater fishes of the Maroni Basin.']
['33892517', 'First Detection of Rabbit Haemorrhagic Disease Virus (RHDV2) in Singapore.']
['33892512', 'Phylogenetic and selection analysis of an expanded family of putatively pore-forming jellyfish toxins (Cnidaria: Medusozoa).']
['33892511', 'Genomic signature of shifts in selection and alkaline adaptation in highland fish.']
['33892498', 'The asgard archaeal-unique contribution to protein families of the eukaryoti